In [2]:
import numpy as np
import re
import nltk
from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model
import tensorflow as tf
from keras.layers import SimpleRNN
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed
from keras.models import Model

Using TensorFlow backend.


In [3]:
#Global Variables declaration and intitialization
INPUT_VECTOR_LENGTH = 20
OUTPUT_VECTORLENGTH = 20
minimum_length = 2
maximum_length = 20
sample_size = 30000 
WORD_START = 1
WORD_PADDING = 0
GLOVE_MODEL = "C:/Users/15712/Downloads/glove.6B/glove.6B.50d.txt"


In [4]:
exit_words = [
        'bye', 'goodbye', 'exit', 
        'tata','see you','terminate',
        'Bye', 'Goodbye', 'Exit',
        'Tata','See you','Terminate'
         ]

In [5]:
#Mapping the Ids to lines and splitting the lines by using the delimiter.
def map_linetoID(movie_lines):
    linetoID_mapping = {}
    for line in movie_lines:
        split_line = line.split(' +++$+++ ')
        if len(split_line) == 5:
            linetoID_mapping[split_line[0]] = split_line[4]
    return linetoID_mapping

In [6]:
#Splitting the converstions by the delimiter and creating a list of coversation ID's.
def extract_converstionIDs(conversation_lines):
    conversations = []
    for line in conversation_lines[:-1]:
        split_line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
        conversations.append(split_line.split(','))
    return conversations

In [7]:
#Function is used to form pairs of questions and answers.
def extract_quesans_pairs(linetoID_mapping,conversations):
    questions = []
    answers = []
    for con in conversations:
        for i in range(len(con)-1):
            questions.append(linetoID_mapping[con[i]])
            answers.append(linetoID_mapping[con[i+1]])
    return questions,answers

In [8]:
#Function is used to transsfrom the text
#For example I'm gets transformed to I am
def transform_text(input_text):
    input_text = input_text.lower()
    input_text = re.sub(r"I'm", "I am", input_text)
    input_text = re.sub(r"he's", "he is", input_text)
    input_text = re.sub(r"she's", "she is", input_text)
    input_text = re.sub(r"it's", "it is", input_text)
    input_text = re.sub(r"that's", "that is", input_text)
    input_text = re.sub(r"what's", "that is", input_text)
    input_text = re.sub(r"where's", "where is", input_text)
    input_text = re.sub(r"how's", "how is", input_text)
    input_text = re.sub(r"\'ll", " will", input_text)
    input_text = re.sub(r"\'ve", " have", input_text)
    input_text = re.sub(r"\'re", " are", input_text)
    input_text = re.sub(r"\'d", " would", input_text)
    input_text = re.sub(r"\'re", " are", input_text)
    input_text = re.sub(r"won't", "will not", input_text)
    input_text = re.sub(r"can't", "cannot", input_text)
    input_text = re.sub(r"n't", " not", input_text)
    input_text = re.sub(r"'til", "until", input_text)
    input_text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", input_text)
    input_text = " ".join(input_text.split())
    return input_text


In [9]:
#Filter the questions and answer. The minimum length is 2 and 
#maximum is 20
def filter_ques_ans(clean_questions,clean_answers):
    # Filter out the questions that are too short/long
    short_questions_temp = []
    short_answers_temp = []
    for i, question in enumerate(clean_questions):
        if len(question.split()) >= minimum_length and len(question.split()) <= maximum_length:
            short_questions_temp.append(question)
            short_answers_temp.append(clean_answers[i])
    short_questions = []
    short_answers = []
    for i, answer in enumerate(short_answers_temp):
        if len(answer.split()) >= minimum_length and len(answer.split()) <= maximum_length:
            short_answers.append(answer)
            short_questions.append(short_questions_temp[i])
    return short_questions,short_answers

In [10]:
#Calculate the word count 
def create_vocabulary(tokenized_ques,tokenized_ans):
    vocabulary = {}
    for question in tokenized_ques:
        for word in question:
            if word not in vocabulary:
                vocabulary[word] = 1
            else:
                vocabulary[word] += 1
    for answer in tokenized_ans:
        for word in answer:
            if word not in vocabulary:
                vocabulary[word] = 1
            else:
                vocabulary[word] += 1  
    return vocabulary

In [11]:
#Create the encodings and decodings by assigning unique 
#index to the words.
def create_encoding_decoding(vocabulary):
    threshold = 15
    count = 0
    for k,v in vocabulary.items():
        if v >= threshold:
            count += 1
    vocab_size  = 2 
    encoding = {}
    decoding = {1: 'START'}
    for word, count in vocabulary.items():
        if count >= threshold:
            encoding[word] = vocab_size 
            decoding[vocab_size ] = word
            vocab_size += 1
    return encoding,decoding,vocab_size

In [12]:
#Convert the training and validation data into vectors
def transform(encoding, data, vector_size=20):
    transformed_data = np.zeros(shape=(len(data), vector_size))
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            try:
                transformed_data[i][j] = encoding[data[i][j]]
            except:
                transformed_data[i][j] = encoding['<UNKNOWN>']
    return transformed_data

In [13]:
#Create glove embeddings form the pre-trained glove model.
def create_gloveEmbeddings(encoding,size):
    file = open(GLOVE_MODEL, mode='rt', encoding='utf8')
    words = set()
    word_to_vec_map = {}
    for line in file:
        line = line.strip().split()
        word = line[0]
        words.add(word)
        word_to_vec_map[word] = np.array(line[1:], dtype=np.float64)
    embedding_matrix = np.zeros((size, 50))
    for word,index in encoding.items():
        try:
            embedding_matrix[index, :] = word_to_vec_map[word.lower()]
        except: continue
    return embedding_matrix

In [14]:
#Creating the LSTM model
def create_model(dict_size,embed_layer,hidden_dim):
    
    encoder_inputs = Input(shape=(maximum_length, ), dtype='int32',)
    encoder_embedding = embed_layer(encoder_inputs)
    encoder_LSTM = LSTM(hidden_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    decoder_inputs = Input(shape=(maximum_length, ), dtype='int32',)
    decoder_embedding = embed_layer(decoder_inputs)
    decoder_LSTM = LSTM(hidden_dim, return_state=True, return_sequences=True)
    decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    outputs = TimeDistributed(Dense(dict_size, activation='softmax'))(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], outputs)
    return model

In [15]:
# predicting the answer to the question
#and returning the output vectors.
def prediction_answer(user_input,model):
    transformed_input = transform_text(user_input)
    input_tokens = [nltk.word_tokenize(transformed_input)]
    input_tokens = [input_tokens[0][::-1]]  #reverseing input seq
    encoder_input = transform(encoding, input_tokens, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_VECTORLENGTH))
    decoder_input[:,0] = WORD_START
    for i in range(1, OUTPUT_VECTORLENGTH):
        pred_output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = pred_output[:,i]
    return pred_output

In [16]:
#decoding the vectors.
def decode_answer(decoding, ans_vec):
    ans = ''
    for i in ans_vec:
        if i == 0:
            break
        ans += ' '
        ans += decoding[i]
    return ans

In [17]:
linetoID_mapping={}
conversations=[]
#Reading the conversational data
movie_lines = open('C:/Users/15712/Downloads/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conversation_lines = open('C:/Users/15712/Downloads/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')
#calling map_linetoID()
linetoID_mapping=map_linetoID(movie_lines)
    
#calling extract_converstions()
conversations=extract_converstionIDs(conversation_lines)
    
#extracting question answer pairs
questions,answers=extract_quesans_pairs(linetoID_mapping,conversations)
transformed_ques = []
for question in questions:
    transformed_ques.append( transform_text(question))
transformed_answers = []    
for answer in answers:
     transformed_answers.append(transform_text(answer))
    
#Limiting the length of questionas and answers
filtered_questions=[]
filtered_answers=[]
filtered_questions,filtered_answers=filter_ques_ans(transformed_ques,transformed_answers)
    
#Tokeninzing
filtered_questions = filtered_questions[:sample_size]
filtered_answers = filtered_answers[:sample_size]
#tokenizing the questions and answers
tokenized_ques = [nltk.word_tokenize(sent) for sent in filtered_questions]
tokenized_ans = [nltk.word_tokenize(sent) for sent in filtered_answers]
    
#Splitting the data into training and validation datasets
size = len(tokenized_ques)
training_input  = tokenized_ques[:round(size*(80/100))]
training_input  = [tr_input[::-1] for tr_input in training_input] #reverseing input seq for better performance
training_output = tokenized_ans[:round(size*(80/100))]

# We will use the remaining for validation
validation_input = tokenized_ques[round(size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = tokenized_ans[round(size*(80/100)):]

print('Number of Samples used for training:', len(training_input))
print('Number of samples in the validation:', len(validation_input))
    
#creating vacabulary
vocabulary={}
vocabulary=create_vocabulary(tokenized_ques,tokenized_ans)
print("Length of vocabulary:", len(vocabulary))
    
#creating encodings and decodings
dict_size=0
encoding={}
decoding={}
encoding,decoding,dict_size=create_encoding_decoding(vocabulary)
dict_size=dict_size+1
decoding[len(encoding)+2] = '<UNKNOWN>'
encoding['<UNKNOWN>'] = len(encoding)+2
print("The size of the dictionary:",dict_size)
print("The size of encoding:",len(encoding))
print("The size of decoding:",len(decoding))
    
    
#Function call to the transform function
encoded_training_input = transform(
encoding, training_input, vector_size=INPUT_VECTOR_LENGTH)
encoded_training_output = transform(
encoding, training_output, vector_size=OUTPUT_VECTORLENGTH)
print('Shape of Encoded Training Input', encoded_training_input.shape)
print('Shape of Encoded Training Output', encoded_training_output.shape)
    
#For Validation data 
encoded_validation_input = transform(
encoding, validation_input, vector_size=INPUT_VECTOR_LENGTH)
encoded_validation_output = transform(
encoding, validation_output, vector_size=OUTPUT_VECTORLENGTH)
print('Shape of Encoded validation Input', encoded_validation_input.shape)
print('Shape of Encoded validation Output', encoded_validation_output.shape)
    
#Create the glove embedding which will be used as weights for the embedding layer.
tf.keras.backend.clear_session()
embedding_matrix = np.zeros((dict_size, 50))
embedding_matrix= create_gloveEmbeddings(encoding,dict_size)
print(embedding_matrix.shape)
    
#forming th embedding layer
embed_layer = Embedding(input_dim=dict_size, output_dim=50, trainable=True,)
embed_layer.build((None,))
embed_layer.set_weights([embedding_matrix])
    
#creating model
hidden_dim=300
lstm_model = create_model(dict_size,embed_layer,hidden_dim)
#getting the summary of model
lstm_model.summary()
    
#compiling the model
lstm_model.compile(optimizer='adam', loss ='categorical_crossentropy', metrics = ['accuracy'])
    
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = WORD_START
training_decoder_output = np.eye(dict_size)[encoded_training_output.astype('int32')]

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = WORD_START
validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype('int32')]
    

    
    
    
   

    

Number of Samples used for training: 24000
Number of samples in the validation: 6000
Length of vocabulary: 16570
The size of the dictionary: 1963
The size of encoding: 1961
The size of decoding: 1962
Shape of Encoded Training Input (24000, 20)
Shape of Encoded Training Output (24000, 20)
Shape of Encoded validation Input (6000, 20)
Shape of Encoded validation Output (6000, 20)
(1963, 50)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20)           0                                            
_________________________________________________________________________________________________

In [17]:
#fitting the model
lstm_model.fit(x=[training_encoder_input, training_decoder_input], y=[training_decoder_output],
    validation_data=([validation_encoder_input, validation_decoder_input], [validation_decoder_output]),
          batch_size=64, epochs=100)

C:\Users\15712\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 24000 samples, validate on 6000 samples
Epoch 1/100
24000/24000 [==============================] - 135s 6ms/step - loss: 2.6099 - accuracy: 0.5640 - val_loss: 2.3026 - val_accuracy: 0.5974
Epoch 2/100
24000/24000 [==============================] - 142s 6ms/step - loss: 2.1739 - accuracy: 0.6089 - val_loss: 2.1605 - val_accuracy: 0.6070
Epoch 3/100
24000/24000 [==============================] - 141s 6ms/step - loss: 2.0650 - accuracy: 0.6194 - val_loss: 2.0866 - val_accuracy: 0.6179
Epoch 4/100
24000/24000 [==============================] - 145s 6ms/step - loss: 1.9997 - accuracy: 0.6263 - val_loss: 2.0451 - val_accuracy: 0.6225
Epoch 5/100
24000/24000 [==============================] - 150s 6ms/step - loss: 1.9559 - accuracy: 0.6295 - val_loss: 2.0221 - val_accuracy: 0.6250
Epoch 6/100
24000/24000 [==============================] - 146s 6ms/step - loss: 1.9216 - accuracy: 0.6320 - val_loss: 2.0041 - val_accuracy: 0.6256
Epoch 7/100
24000/24000 [==============================] 

In [18]:
lstm_model.save('lstm_model_glove_embeddings.h5')

In [26]:
from keras.models import load_model
lstm_model=load_model('lstm_model_glove_embeddings.h5')

C:\Users\15712\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [27]:
#Printing the sample Questions and answers.
for i in range(20):
    output = prediction_answer(filtered_questions[i],lstm_model)
    print ('Question:', filtered_questions[i])
    print ('Answer:', decode_answer(decoding, output[0]))
    i=i+1

Question: well, i thought we would start with pronunciation, if that is okay with you.
Answer:  not no general
Question: not the hacking and gagging and spitting part. please.
Answer:  okay tell sir
Question: you are asking me out. that is so cute. that is your name again?
Answer:  forget . it it
Question: gosh, if only we could find kat a boyfriend...
Answer:  let you i
Question: c'esc ma tete. this is my head
Answer:  right there later
Question: that is because it is such a nice one.
Answer:  forget . lord
Question: how is our little find the wench a date plan progressing?
Answer:  well i i
Question: you have my word. as a gentleman
Answer:  you are know
Question: how do you get your hair to look like that?
Answer:  <UNKNOWN> quite it
Question: sure have.
Answer:  i just , do should
Question: i really, really, really wanna go, but i cannot. not unless my sister goes.
Answer:  i i ,
Question: so that is the kind of guy she likes? pretty ones?
Answer:  who i yes course
Question: you kn

In [20]:
print("Hello! I am a Generative Chatbot Trained on Cornell Movie Corpus Data")
user_input = input("~")
predicted_seq =  prediction_answer(user_input,lstm_model)
print (decode_answer(decoding, predicted_seq[0]))
while user_input not in exit_words:
    try:
        user_input = input("~")
        if user_input not in exit_words:
            decode_seq = prediction_answer(user_input,lstm_model)
            print (decode_answer(decoding, decode_seq[0]))
        else:
            break
    except EOFError:
        print("Bye")
        exit()

Hello! I am a Generative Chatbot Trained on Cornell Movie Corpus Data
~Hello
 when not ,
~How are you doing?
 not that all
~Where are you going?
 i 'm got
~Should we go out?
 no not . i i i
~Why not?
 i this ...
~I want some food
 i do harry
~Where is the pen
 i . there
~when is the flight ?
 the hotel ya
~shall we go out tomorrow?
 yes , ,
~Are you not well?
 yes i do yes
~Let us party today.
 oh do do
~I want to go to paris.
 thank who then
~can you come with me?
 i i do
~Let us go then
 right wait hello
~pack your bags.
 you baby you i
~Who all are coming?
 pardon 'm i
~bye
